In [5]:
import numpy as np
import scipy.stats as scs
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf
import matplotlib as mpl
import datetime as dt
import typing
import scipy.stats as stats
from datetime import datetime, timedelta
plt.style.use('seaborn')
%matplotlib inline

In [11]:
sales = [10500,5000,6500,9500,8500,11000]
profit = [1.6,3,2.5,1.75,2.5,1.50]
product = ['A','B','C','D','E','F']
DF = pd.DataFrame([sales,profit],columns = product,index = ['Sales','Profit']).T
DF['Sales_Profit'] = DF['Sales']*DF['Profit']
DF.sort_values(by = 'Sales_Profit',ascending=False)


,Sales,Profit,Sales_Profit
E,8500.0,2.50,21250.0
A,10500.0,1.60,16800.0
D,9500.0,1.75,16625.0
F,11000.0,1.50,16500.0
C,6500.0,2.50,16250.0
B,5000.0,3.00,15000.0


In [3]:
stats.norm.cdf(0.36867)*1000 - 1000*(np.exp(-0.06*3))*stats.norm.cdf(0.12619)

184.23975842658803

In [5]:
stats.norm.pdf(.045473)

0.39853002837836904

In [6]:
google = pd.DataFrame(yf.download('googl', start = '2022-12-01')['Adj Close']).pct_change().dropna()

[*********************100%***********************]  1 of 1 completed


In [7]:
d = pd.date_range(start='2023-02-02', end = '2023-02-17', freq='D')
len(d)

16

In [8]:
start = '2023-02-02'
end = '2023-02-17'
date_range = pd.date_range(start = start, end = end , freq = 'B')
delta = date_range[-1] - date_range[0]
print("Time delta:", delta)

Time delta: 15 days 00:00:00


# Option Formula

## Call Option 

- Gives the owner the right to buy but not the obligation to buy the underlying asset at a specific price $K$.

$C_{0,t} = S_0 \text{N}({d_1}) - Ke^{-rT} \text{N}({d_2})$

## Put Option

$P_{0,t} = Ke^{-rT} \text{N}({-d_2}) - S_0 \text{N}({d_1})$


In [14]:
def call(S0, K, r, sigma, T):
    """
    Returns the Black-Scholes Call Option Price
    S0: Spot price at time 0
    K: Strike price
    r: risk-free interest rate
    sigma: annualized volatility of the option
    T: Time to maturity

    """
    d1 = (np.log(S0/K)+(r+((sigma**2)/2))*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    pv = K*np.exp(-r*T)

    C = S0*(stats.norm.cdf(d1)) - pv*(stats.norm.cdf(d2))
    return pd.DataFrame([np.round(C,4)], columns = ['Call Option'],index = ['Price'] )
def put(S0, K, r, sigma, T):
    """
    Returns the Black-Scholes Put Option Price
    S0: Spot price at time 0
    K: Strike price
    r: risk-free interest rate
    sigma: annualized volatility of the option
    T: Time to maturity

    """
    d1 = (np.log(S0/K)+(r+((sigma**2)/2))*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    pv = K*np.exp(-r*T)

    C = pv*(stats.norm.cdf(-d2)) - S0*(stats.norm.cdf(-d1))
    return pd.DataFrame([np.round(C,4)], columns = ['Put Option'],index = ['Price'] )
    

In [10]:
call(100,105,.04,.3,.25)

,Call Option
Price,4.3215


In [15]:
put(100,105,.04,.3,.25)

,Put Option
Price,8.2767


In [12]:
print(call.__doc__)


    Returns the Black-Scholes Call Option Price
    S0: Spot price at time 0
    K: Strike price
    r: risk-free interest rate
    sigma: annualized volatility of the option
    T: Time to maturity

    


In [12]:
def option(type = None, So = None, K = None, r = None, sigma = None, current_date = None, expiration_date = None):
    no_business_days = np.busday_count(begindates = current_date, enddates = expiration_date)
    Time = (no_business_days/252)
    # Divide by 252 because there are 252 trading days
    d1 = (np.log(So/K)+(r+((sigma**2)/2))*Time)/(sigma*np.sqrt(Time))
    d2 = d1 - sigma*np.sqrt(Time)
    present_value = K*(np.exp(-r*(Time)))
    if type == 'Call':
        price = So*(scs.norm.cdf(d1))-present_value*(scs.norm.cdf(d2))
    elif type == "Put":
        price = present_value*(scs.norm.cdf(-d2))-So*(scs.norm.cdf(-d1))
    else:
        print("Wrong security name")
    return pd.DataFrame([np.round(price,4)],columns = ['Option Price'])
        
        

In [21]:
option('Put',So = 444.49,K = 445, r = 0.05388,sigma=.1547,current_date='2023-08-11',expiration_date='2023-10-20')

,Option Price
0,10.1643


In [13]:
def fx_options(spot_fx_rate: float,strike_fx_rate:float,sigma: float ,domestic_interest: float,foreign_interest:float,domestic_currency:str, foreign_currency:str, spot_date:str,expiration_date: str, option_type:str):
    tau = len(pd.date_range(start=spot_date,end = expiration_date))
    if domestic_currency == 'USD':
        t1 = tau/360
    else:
        t1 = tau/365
    if foreign_currency == 'USD':
        t2 = tau/360
    else:
        t2 = tau/365
    domestic_discount = np.exp(-domestic_interest*t1)
    foreign_discount = np.exp(-foreign_interest*t2)
    d1 =(np.log(spot_fx_rate/strike_fx_rate)+((domestic_interest-foreign_interest)+((sigma**2)/2))*tau)/(sigma*np.sqrt(tau))
    d2 = d1 - sigma*np.sqrt(tau)

    if option_type == 'Call':
        price = spot_fx_rate*foreign_discount*(scs.norm.cdf(d1)) - strike_fx_rate*domestic_discount*(scs.norm.cdf(d2))

    elif option_type == 'Put':
        price = strike_fx_rate*domestic_discount*(scs.norm.cdf(-d2)) -spot_fx_rate*foreign_discount*(scs.norm.cdf(-d1))
    else:
        print('Wrong Option Type')

    return np.round(price,6)


    


In [14]:
fx_options(1.6,1.170,0.05,.0475,.0475,'EUR','GBP','2023-02-27','2023-02-28','Call')

0.429888

In [15]:
scs.norm.cdf(0)

0.5

In [16]:
option('Call', So = 106.7,K = 95,r = .04,sigma = .5, current_date='2023-02-02',expiration_date='2023-02-17')

,Option Price
0,12.5464


In [17]:
option('Call', So = 112,K = 95,r = .0468,sigma = .5, current_date='2023-02-03',expiration_date='2023-02-17')

,Option Price
0,17.3773


In [18]:
scs.norm.cdf(-.206)

0.4183954556526073